In [4]:
import xarray as xr
import numpy as np
from pathlib import Path

def check_nc_resolution(nc_path, var_name):
    ds = xr.open_dataset(nc_path)
    
    # Find lat/lon variable names (works even if they are 'lat'/'lon')
    lat_name = [name for name in ds.coords if 'lat' in name.lower()][0]
    lon_name = [name for name in ds.coords if 'lon' in name.lower()][0]
    
    lat = ds[lat_name].values
    lon = ds[lon_name].values
    
    # Handle descending latitudes
    lat_res = np.abs(np.mean(np.diff(lat)))
    lon_res = np.abs(np.mean(np.diff(lon)))
    
    print(f"File: {nc_path}")
    print(f"Variable: {var_name}")
    print(f"Latitude spacing: {lat_res:.6f} degrees")
    print(f"Longitude spacing: {lon_res:.6f} degrees")
    
    lat_res_m = lat_res * 111_000  # m per degree latitude
    lon_res_m = lon_res * 111_000 * np.cos(np.deg2rad(np.mean(lat)))
    
    print(f"Approx pixel size: {lat_res_m:.2f} m (N-S), {lon_res_m:.2f} m (E-W)")

# Example usage
era5_nc = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc")
check_nc_resolution(era5_nc, var_name='t2m')


File: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc
Variable: t2m
Latitude spacing: 0.250000 degrees
Longitude spacing: 0.250000 degrees
Approx pixel size: 27750.00 m (N-S), 24032.20 m (E-W)


In [7]:
import rasterio
from pathlib import Path

def check_pixel_size_tif_precise(tif_path):
    with rasterio.open(tif_path) as src:
        transform = src.transform
        pixel_width = transform.a
        pixel_height = -transform.e  # make positive
        crs = src.crs
        
        print(f"File: {tif_path}")
        print(f"CRS: {crs}")
        print(f"Pixel size (width): {pixel_width:.8f} {'m' if crs.is_projected else 'degrees'}")
        print(f"Pixel size (height): {pixel_height:.8f} {'m' if crs.is_projected else 'degrees'}")

# Example usage
lulc_file = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif")
check_pixel_size_tif_precise(lulc_file)


File: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif
CRS: EPSG:4326
Pixel size (width): 0.00416667 degrees
Pixel size (height): 0.00416667 degrees


In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pathlib import Path

def reproject_resample_to_30m(input_path, output_path):
    # Step 1: Open input file
    with rasterio.open(input_path) as src:
        print(f"Original CRS: {src.crs}")
        print(f"Original resolution: {src.res}")

        # Step 2: Choose a projected CRS (UTM based on center of dataset)
        utm_crs = src.crs
        if utm_crs.is_geographic:
            # Auto-pick a UTM zone based on dataset center
            from pyproj import CRS
            lon, lat = src.xy(src.height // 2, src.width // 2)
            utm_crs = CRS.from_user_input(f"+proj=utm +zone={(int((lon + 180) / 6) % 60) + 1} +datum=WGS84 +units=m +no_defs")
            print(f"Target UTM CRS: {utm_crs}")

        # Step 3: Calculate transform for 30 m resolution
        transform, width, height = calculate_default_transform(
            src.crs, utm_crs, src.width, src.height, *src.bounds, resolution=30
        )

        # Step 4: Define output profile
        profile = src.profile.copy()
        profile.update({
            'crs': utm_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Step 5: Reproject + resample
        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=utm_crs,
                    resampling=Resampling.bilinear  # or nearest for categorical (like LULC)
                )

    print(f"Saved reprojected 30 m raster → {output_path}")

# Example usage:
input_file = Path(r"C:\path\to\your\LULC_2015_clipped.tif")  # Change path
output_file = input_file.with_name(input_file.stem + "_30m.tif")
reproject_resample_to_30m(input_file, output_file)
